In [1]:
import os
os.path.abspath("./")

'/home/yuxiangliao/PhD'

# Spacy supported functions

## Part-os-speech tags

In [2]:
import spacy
nlp = spacy.load("en_core_sci_sm")
doc = nlp("There is no focal consolidation, pleural effusion or pneumothorax.  Bilateral nodular opacities that most likely represent nipple shadows. The cardiomediastinal silhouette is normal.  Clips project over the left lung, potentially within the breast. The imaged upper abdomen is unremarkable. Chronic deformity of the posterior left sixth and seventh ribs are noted.")
for token in doc:
    # Print the text and the predicted part-of-speech tag
    print(token.text, token.pos_)

There PRON
is AUX
no DET
focal ADJ
consolidation NOUN
, PUNCT
pleural ADJ
effusion NOUN
or CCONJ
pneumothorax NOUN
. PUNCT
  SPACE
Bilateral ADJ
nodular ADJ
opacities NOUN
that SCONJ
most ADV
likely ADV
represent VERB
nipple NOUN
shadows NOUN
. PUNCT
The DET
cardiomediastinal ADJ
silhouette NOUN
is AUX
normal ADJ
. PUNCT
  SPACE
Clips PROPN
project NOUN
over ADP
the DET
left ADJ
lung NOUN
, PUNCT
potentially ADV
within ADP
the DET
breast NOUN
. PUNCT
The DET
imaged VERB
upper ADJ
abdomen NOUN
is AUX
unremarkable ADJ
. PUNCT
Chronic ADJ
deformity NOUN
of ADP
the DET
posterior ADJ
left ADJ
sixth ADJ
and CCONJ
seventh ADJ
ribs NOUN
are AUX
noted VERB
. PUNCT


## Syntactic dependencies

In [3]:
import spacy
nlp = spacy.load("en_core_sci_sm")
doc = nlp("There is no focal consolidation, pleural effusion or pneumothorax.  Bilateral nodular opacities that most likely represent nipple shadows. The cardiomediastinal silhouette is normal.  Clips project over the left lung, potentially within the breast. The imaged upper abdomen is unremarkable. Chronic deformity of the posterior left sixth and seventh ribs are noted.")
for token in doc:
    print(token.text, token.pos_, token.dep_, token.head.text)

There PRON expl is
is AUX ROOT is
no DET det consolidation
focal ADJ amod consolidation
consolidation NOUN attr is
, PUNCT punct consolidation
pleural ADJ amod effusion
effusion NOUN conj consolidation
or CCONJ cc effusion
pneumothorax NOUN conj effusion
. PUNCT punct is
  SPACE nummod opacities
Bilateral ADJ amod opacities
nodular ADJ dep Bilateral
opacities NOUN ROOT opacities
that SCONJ mark represent
most ADV advmod likely
likely ADV advmod represent
represent VERB acl:relcl opacities
nipple NOUN compound shadows
shadows NOUN dobj represent
. PUNCT punct opacities
The DET det silhouette
cardiomediastinal ADJ amod silhouette
silhouette NOUN nsubj normal
is AUX cop normal
normal ADJ ROOT normal
. PUNCT punct normal
  SPACE compound Clips
Clips PROPN compound project
project NOUN ROOT project
over ADP case lung
the DET det lung
left ADJ amod lung
lung NOUN nmod project
, PUNCT punct project
potentially ADV advmod breast
within ADP case breast
the DET det breast
breast NOUN nmod projec

## Named entities

In [4]:
import spacy
nlp = spacy.load("en_core_sci_sm")
doc = nlp("There is no focal consolidation, pleural effusion or pneumothorax.  Bilateral nodular opacities that most likely represent nipple shadows. The cardiomediastinal silhouette is normal.  Clips project over the left lung, potentially within the breast. The imaged upper abdomen is unremarkable. Chronic deformity of the posterior left sixth and seventh ribs are noted.")
for ent in doc.ents:
    # Print the entity text and its label
    print(ent.text, ent.label_)

focal consolidation ENTITY
pleural effusion ENTITY
pneumothorax ENTITY
Bilateral ENTITY
nodular opacities ENTITY
nipple shadows ENTITY
cardiomediastinal silhouette ENTITY
Clips project ENTITY
left lung ENTITY
breast ENTITY
imaged ENTITY
upper ENTITY
abdomen ENTITY
Chronic deformity ENTITY
posterior left sixth ENTITY
ribs ENTITY


In [5]:
import scispacy
import spacy
nlp = spacy.load("en_core_sci_scibert")
doc = nlp("There is no focal consolidation, pleural effusion or pneumothorax.  Bilateral nodular opacities that most likely represent nipple shadows. The cardiomediastinal silhouette is normal.  Clips project over the left lung, potentially within the breast. The imaged upper abdomen is unremarkable. Chronic deformity of the posterior left sixth and seventh ribs are noted.")
for ent in doc.ents:
    # Print the entity text and its label
    print(ent.text, ent.label_)

pleural effusion ENTITY
pneumothorax ENTITY
Bilateral ENTITY
nodular ENTITY
nipple shadows ENTITY
cardiomediastinal silhouette ENTITY
Clips project ENTITY
left lung ENTITY
breast ENTITY
imaged ENTITY
upper abdomen ENTITY
Chronic deformity ENTITY
posterior left sixth ENTITY
ribs ENTITY


/home/yuxiangliao/anaconda3/lib/python3.9/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [3]:
import scispacy
import spacy
nlp = spacy.load("en_core_sci_lg")
doc = nlp("There is no focal consolidation, pleural effusion or pneumothorax.  Bilateral nodular opacities that most likely represent nipple shadows. The cardiomediastinal silhouette is normal.  Clips project over the left lung, potentially within the breast. The imaged upper abdomen is unremarkable. Chronic deformity of the posterior left sixth and seventh ribs are noted.")
for ent in doc.ents:
    # Print the entity text and its label
    print(ent.text, ent.label_)

focal consolidation ENTITY
pleural effusion ENTITY
pneumothorax ENTITY
Bilateral ENTITY
nodular opacities ENTITY
nipple shadows ENTITY
cardiomediastinal silhouette ENTITY
left lung ENTITY
breast ENTITY
imaged ENTITY
upper abdomen ENTITY
Chronic deformity ENTITY
posterior left sixth ENTITY
ribs ENTITY


## Pipeline

In [ ]:
import scispacy
import spacy
nlp = spacy.load("en_core_sci_lg")
# 关闭词性标注器tagger和依存关系标注器parser
with nlp.select_pipes(disable=["tagger", "parser"]):
    print(nlp.pipeline)
print(nlp.pipeline)

# NeuralCoref

https://github.com/explosion/coreferee

In [ ]:
# !python3 -m pip install coreferee
# !python3 -m coreferee install en
# !python -m spacy download en_core_web_lg

In [53]:
text = """Lung volumes are low.  This results in crowding of the bronchovascular structures.  There may be mild pulmonary vascular congestion.  The heart size is borderline enlarged.  The mediastinal and hilar contours are relatively unremarkable.  Innumerable nodules are demonstrated in both lungs, more pronounced in the left upper and lower lung fields compatible with metastatic disease.  No new focal consolidation, pleural effusion or pneumothorax is seen, with chronic elevation of right hemidiaphragm again seen.  The patient is status post right lower lobectomy.  Rib deformities within the right hemithorax is compatible with prior postsurgical changes.
"""

In [54]:
import coreferee, spacy

nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('coreferee')
doc = nlp(text)
doc._.coref_chains.print()